In [1]:
# default_exp pdf_generator

# pdf_generator

> Ce module permet de lire les fichiers reçu et de générer le PDF, à l'aide du module pdf_helper.

In [32]:
#hide
from nbdev.showdoc import *
!pip install -q -r requirements.txt
# For multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [51]:
#export
import datetime
import pandas as pd
#from fpdf import FPDF
from covid_certificate_generator import pdf_helper
import io, sys, os
import tempfile
import pathlib

In [34]:
%%writefile data.csv
NomParent,PrenomParent,NomEnfant,PrenomEnfant,DateNaissance,Moyen
NomParent1,PrenomParent1,NomEnfant1,PrenomEnfant1,12/06/2010,pied
NomParent2,PrenomParent2,NomEnfant2,PrenomEnfant2,12/04/2011,pied
NomParent3,PrenomParent3,NomEnfant3,PrenomEnfant3,12/04/2011,vélo
NomParent4,PrenomParent4,NomEnfant4,"Prenom, Enfant4",12/04/2011,car

Overwriting data.csv


In [35]:
#ignore
df = pd.read_csv('data.csv')
df

,NomParent,PrenomParent,NomEnfant,PrenomEnfant,DateNaissance,Moyen
0,NomParent1,PrenomParent1,NomEnfant1,PrenomEnfant1,12/06/2010,pied
1,NomParent2,PrenomParent2,NomEnfant2,PrenomEnfant2,12/04/2011,pied
2,NomParent3,PrenomParent3,NomEnfant3,PrenomEnfant3,12/04/2011,vélo
3,NomParent4,PrenomParent4,NomEnfant4,"Prenom, Enfant4",12/04/2011,car


In [82]:
#export
class PDFGenerator:
    config = None # Dict configuration
    school = None
    school_sign = None
    pdf_w=210
    pdf_h=297
    
    def __init__(self, config_file=None):
        """
        Constructor
        :param config_file: str, Path to config file
        :return:
        """
    def generate_one_attestation(self, prenom_parent, nom_parent, prenom_enfant, nom_enfant, date_naissance, moyen):
        self.pdf.add_page()
        #pdf.lines()
        self.pdf.titles()
        self.pdf.texte_parent(prenom_parent, nom_parent)
        self.pdf.texte_enfant(prenom_enfant, nom_enfant,date_naissance, moyen)
        self.pdf.texte_etablissement(
            self.school['school_name'], self.school['school_adress'], self.school_sign, self.school['city'])

    def generate(self, row):
        self.generate_one_attestation(row.PrenomParent, row.NomParent, row.PrenomEnfant, row.NomEnfant, row.DateNaissance, row.Moyen)

        
    def get_pdf(self, students_filename, students_data, school_sign, school):
#         students_file = io.BytesIO(students_file)
        self.school_sign = school_sign
        self.school = school
        try:
            if '.csv' in students_filename:
                # Assume that the user uploaded a CSV file
                df = pd.read_csv(students_data)
                if len(df.columns) == 1: # Is separator , ?
                    df = pd.read_csv(io.BytesIO(students_data.getbuffer()), sep=';')
                    if len(df.columns) == 1: # Is separator , ?
                        df = pd.read_csv(io.BytesIO(students_data.getbuffer()), sep='\t')
            else:
                # Assume that the user uploaded an excel file
                df = pd.read_excel(students_data) #io.BytesIO(decoded)
        except:
            error = f'ERREUR à la lecture du fichier {students_filename}: {sys.exc_info()[0]}'
            print(error)
            raise
            
        if len(df.columns) < 6:
            raise ValueError("Format de fichier invalide : pas assez de colonnes !")
        self.pdf = pdf_helper.PDF(orientation='P', unit='mm', format='A4')
        self.pdf.set_author('Data For Good France')
        _ = df.apply(self.generate, axis=1)

    def get_pdf_from_file(self, students_filename, school_sign, school, output_name):
        # Read file
        with open(students_filename, 'rb') as fh:
            students_data = io.BytesIO(fh.read())
        self.get_pdf(students_filename, students_data, school_sign, school)
        _ = self.pdf.output(output_name,'F')

    def get_temp_file(self, ext=None):
        tmp_dir = tempfile._get_default_tempdir()
        tmp_name = next(tempfile._get_candidate_names())
        if ext:
            return os.path.join(tmp_dir, tmp_name + ext)
        else:
            return os.path.join(tmp_dir, tmp_name)
    def get_pdf_from_BytesIO(self, students_filename, students_data, school_sign_filename, school_sign_data, school):
        school_sign_filename = self.get_temp_file(school_sign_filename)
        with open(school_sign_filename, "wb") as f:
            f.write(school_sign_data.getbuffer())
        self.get_pdf(students_filename, students_data, school_sign_filename, school)
        output_name = self.get_temp_file('.pdf')
        _ = self.pdf.output(output_name,'F')
        # Clean temp file for school image
        path = pathlib.Path(school_sign_filename)
        path.unlink()
        return output_name
        #return self.pdf.output(dest='I')


In [83]:
%%time
school={
    'school_name':'Ecole Jean Jaurès',
    'school_adress':'rue Jean Jaurès, 42 000 Libreville',
    'city':'Libreville'
}
students_filename = 'data.csv'
school_sign_filename = 'cachet.jpg'

#decoded = base64.b64decode(students_file)
#students_file = io.BytesIO(decoded)


with open(students_filename, 'rb') as fh:
    students_data = io.BytesIO(fh.read())
with open(school_sign_filename, 'rb') as fh:
    school_sign_data = io.BytesIO(fh.read())
school_pdf = PDFGenerator()
res = school_pdf.get_pdf_from_BytesIO(students_filename, students_data, school_sign_filename, school_sign_data, school)
type(res)
print(res)
# with open("pdf-bytesio.pdf", "wb") as f:
#     f.write(res.encode())

/tmp/zltucl7f.pdf
CPU times: user 10.5 ms, sys: 1.44 ms, total: 11.9 ms
Wall time: 9.71 ms


In [84]:
%%time

students_file = 'data.csv'
school_sign = 'cachet.jpg'
school_pdf = PDFGenerator()
school_pdf.get_pdf_from_file(students_file, school_sign, school,'test.pdf')

CPU times: user 6.67 ms, sys: 374 µs, total: 7.04 ms
Wall time: 6.24 ms


In [85]:
students_file = './assets/Fichier des élèves.csv'
with open(students_file, 'rb') as fh:
 students_data = io.BytesIO(fh.read())
type(students_data)
copy = students_data
students_data.getbuffer().nbytes
df = pd.read_csv(students_data, sep=';')
df = pd.read_csv(io.BytesIO(students_data.getbuffer()), sep=';')
#df
students_data.getbuffer().nbytes

# # Assume that the user uploaded a CSV file
# df = pd.read_csv(copy)
# if len(df.columns) == 1: # Is separator , ?
#     df = pd.read_csv(students_data, sep=';')
#     if len(df.columns) == 1: # Is separator , ?
#         df = pd.read_csv(students_data, sep='\t')
df

_io.BytesIO

19824

19824

,NomParent,PrenomParent,NomEnfant,PrenomEnfant,DateNaissance,Moyen
0,PIERRE,Pierre,PIERRE,Amande,30/09/12,Pied
1,PIERRET,Pierre,PIERRE,Amande,01/10/12,Pied
2,PIERRICk,Pierre,PIERRE,Amande,02/10/12,Pied
3,PIERREYY,Pierre,PIERRE,Amande,03/10/12,Pied
4,PIERRE 2,Pierre 2,PIERRE 2,Amande 2,04/10/12,Pied
...,...,...,...,...,...,...
338,PIERRE 336,Pierre 336,PIERRE 336,Amande 336,03/09/13,Pied
339,PIERRE 337,Pierre 337,PIERRE 337,Amande 337,04/09/13,Pied
340,PIERRE 338,Pierre 338,PIERRE 338,Amande 338,05/09/13,Pied
341,PIERRE 339,Pierre 339,PIERRE 339,Amande 339,06/09/13,Pied


In [86]:
%%time
students_file = './assets/Fichier des élèves.csv'
school_pdf = PDFGenerator()
school_pdf.get_pdf_from_file(students_file, school_sign, school,'test-csv.pdf')

CPU times: user 109 ms, sys: 0 ns, total: 109 ms
Wall time: 110 ms


In [87]:
%%time
students_file = './assets/Fichier des élèves.xls'
school_pdf = PDFGenerator()
school_pdf.get_pdf_from_file(students_file, school_sign, school,'test-xls.pdf')

CPU times: user 120 ms, sys: 1.48 ms, total: 121 ms
Wall time: 123 ms


In [88]:
%%time
students_file = './assets/Fichier des élèves.xlsx'
school_pdf = PDFGenerator()
school_pdf.get_pdf_from_file(students_file, school_sign, school,'test-xlsx.pdf')

CPU times: user 148 ms, sys: 0 ns, total: 148 ms
Wall time: 147 ms
